In [1]:
import pandas as pd
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize,sent_tokenize
import nltk
import os
import sys  # Required for sys.exit()
from IPython.display import display
from tqdm.notebook import tqdm


/home/arda/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
sec_dict = pd.read_csv("../Data/Loughran-McDonald_MasterDictionary_1993-2023.csv")
sec_dict = sec_dict[~((sec_dict["Source"] == "12of12inf") & (sec_dict["Positive"] == 0) & (sec_dict["Negative"] == 0))]
sec_dict = sec_dict["Word"].str.lower()
sec_dict

9             abandon
10          abandoned
11         abandoning
12        abandonment
13       abandonments
             ...     
86116         wrongly
86138       xenograft
86238         yearend
86239        yearends
86440         zeolite
Name: Word, Length: 7640, dtype: object

In [3]:
original_data = pd.read_csv("../Data/longmerged_deneme_51.csv")
original_data

,Unnamed: 0,Date,OFTIC,PENDS,MEAN,STDEV,BPS,CPS,CPX,CSH,...,commodity_trade_Close_quarterly_return,C_Discretionary_Close_quarterly_return,C_Staples_Close_quarterly_return,Energy_Close_quarterly_return,Financials_Close_quarterly_return,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return
0,0,2011-06-30,AAPL,2011-06-30,17.869831,1.698451,2.6718,0.422900,777.000000,0.209015,...,0.062957,2.970550,4.378340,-5.517243,-6.345335,7.308964,-1.141483,-1.381423,-1.624192,5.051769
1,1,2011-09-30,AAPL,2011-09-30,17.834875,3.131120,2.9872,0.397600,1645.000000,0.258659,...,0.120922,-13.305144,-5.027217,-22.349038,-23.061887,-10.695185,-21.535988,-8.171207,-25.425447,0.418158
2,2,2011-12-31,AAPL,2011-12-31,21.771714,3.087308,3.4500,0.665700,1321.000000,0.249095,...,-0.087046,11.933447,9.541476,18.150742,10.076208,9.328708,15.503083,7.838985,14.100815,7.019633
3,3,2012-03-31,AAPL,2012-03-31,26.426070,2.380069,3.9154,0.528200,1457.000000,0.153558,...,-0.191850,15.556124,4.893814,3.789965,21.538450,8.417417,10.874069,18.506872,10.358213,-2.612559
4,4,2012-06-30,AAPL,2012-06-30,27.652509,3.109565,4.2582,0.384300,2056.000000,0.204900,...,0.484614,-2.905304,2.024644,-7.498254,-7.341768,1.063541,-4.676644,-4.741380,-4.544226,5.565071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14070,14070,2022-09-30,ZION,2022-09-30,58.423077,5.838994,28.4500,1.149000,52.000000,0.622359,...,0.255264,3.615072,-7.550564,0.713179,-3.465819,-5.559891,-5.152279,-6.560733,-7.595104,-6.587759
14071,14071,2022-12-31,ZION,2022-12-31,52.444444,8.477723,29.9500,1.211824,36.000000,0.568366,...,0.110342,-9.329585,11.718866,21.452382,12.648222,12.170758,18.553843,4.765115,14.218494,7.617154
14072,14072,2023-03-31,ZION,2023-03-31,35.441176,4.606920,32.0300,0.190405,48.919936,0.512905,...,-0.040221,15.778870,0.214616,-5.304675,-5.994150,-4.703717,3.024133,21.351649,3.836256,-3.985812
14073,14073,2023-06-30,ZION,2023-06-30,35.980769,4.682907,32.6900,0.225432,52.265163,0.570109,...,0.020449,13.554905,-0.722796,-2.004109,4.852247,2.525868,6.068392,15.131453,2.739895,-3.323977


In [4]:
original_data.columns
original_data = original_data[original_data["Sector"] == "Financials"]
original_data

,Unnamed: 0,Date,OFTIC,PENDS,MEAN,STDEV,BPS,CPS,CPX,CSH,...,commodity_trade_Close_quarterly_return,C_Discretionary_Close_quarterly_return,C_Staples_Close_quarterly_return,Energy_Close_quarterly_return,Financials_Close_quarterly_return,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return
50,50,2011-03-31,AFL,2011-03-31,32.454545,2.411081,11.7900,1.392917,0.245076,0.232178,...,-0.785838,-75.742328,-56.517946,-11.771213,-59.860176,-74.281498,-62.842771,-84.102970,-49.051561,-45.918886
51,51,2011-06-30,AFL,2011-06-30,27.546538,3.568238,12.8250,1.587282,0.259769,0.235599,...,0.062957,2.970550,4.378340,-5.517243,-6.345335,7.308964,-1.141483,-1.381423,-1.624192,5.051769
52,52,2011-09-30,AFL,2011-09-30,26.911765,2.682322,13.6250,1.763514,0.324273,0.310576,...,0.120922,-13.305144,-5.027217,-22.349038,-23.061887,-10.695185,-21.535988,-8.171207,-25.425447,0.418158
53,53,2011-12-31,AFL,2011-12-31,27.750000,1.982881,14.4800,1.621515,0.320066,0.322786,...,-0.087046,11.933447,9.541476,18.150742,10.076208,9.328708,15.503083,7.838985,14.100815,7.019633
54,54,2012-03-31,AFL,2012-03-31,28.115385,2.550767,14.5950,1.660416,0.201505,0.197983,...,-0.191850,15.556124,4.893814,3.789965,21.538450,8.417417,10.874069,18.506872,10.358213,-2.612559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14070,14070,2022-09-30,ZION,2022-09-30,58.423077,5.838994,28.4500,1.149000,52.000000,0.622359,...,0.255264,3.615072,-7.550564,0.713179,-3.465819,-5.559891,-5.152279,-6.560733,-7.595104,-6.587759
14071,14071,2022-12-31,ZION,2022-12-31,52.444444,8.477723,29.9500,1.211824,36.000000,0.568366,...,0.110342,-9.329585,11.718866,21.452382,12.648222,12.170758,18.553843,4.765115,14.218494,7.617154
14072,14072,2023-03-31,ZION,2023-03-31,35.441176,4.606920,32.0300,0.190405,48.919936,0.512905,...,-0.040221,15.778870,0.214616,-5.304675,-5.994150,-4.703717,3.024133,21.351649,3.836256,-3.985812
14073,14073,2023-06-30,ZION,2023-06-30,35.980769,4.682907,32.6900,0.225432,52.265163,0.570109,...,0.020449,13.554905,-0.722796,-2.004109,4.852247,2.525868,6.068392,15.131453,2.739895,-3.323977


In [5]:
finance_ticker = original_data["OFTIC"].values
finance_ticker

array(['AFL', 'AFL', 'AFL', ..., 'ZION', 'ZION', 'ZION'], dtype=object)

In [6]:
# def split_text_into_segments(text, max_words=300):
#     text = re.sub(r'\s+', ' ', text).lower() 
#     sentences = text.split('.')
#     segments = []
#     current_segment = []
#     word_count = 0
#     for sentence in sentences:
#         words = sentence.split()
#         if word_count + len(words) <= max_words:
#             current_segment.append(sentence)
#             word_count += len(words)
#         else:
#             segments.append('. '.join(current_segment) + '.')
#             current_segment = [sentence]
#             word_count = len(words)
#     if current_segment:
#         segments.append('. '.join(current_segment) + '.')
#     return segments

In [7]:

# tokenizer = BertTokenizer.from_pretrained("yiyanghkust/finbert-tone")

# def split_text_into_segments(text, max_tokens=512):
#     max_tokens -= 2  # Adjust for [CLS] and [SEP] tokens

#     sentences = sent_tokenize(text.lower())
#     segments = []
#     current_segment = []
#     token_count = 0

#     for sentence in sentences:
#         tokens = tokenizer.tokenize(sentence)
#         if token_count + len(tokens) <= max_tokens:
#             current_segment.extend(tokens)
#             token_count += len(tokens)
#         else:
#             segments.append(tokenizer.convert_tokens_to_string(current_segment))
#             current_segment = tokens
#             token_count = len(tokens)

#     if current_segment:
#         segments.append(tokenizer.convert_tokens_to_string(current_segment))

#     return segments

# # Example usage
# text = "This is a sample text. It contains multiple sentences. Each sentence is separated by a period."
# segments = split_text_into_segments(text, max_tokens=15)
# print(segments)


In [8]:
# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("yiyanghkust/finbert-tone")

def custom_split_text(text, max_length=508):
    max_length -= 2  # Reserve space for special tokens like [CLS] and [SEP]

    # Tokenize text into sentences
    sentences = sent_tokenize(text)
    segments = []
    current_segment = []
    current_length = 0

    # Process each sentence
    for sentence in sentences:
        tokens = tokenizer.tokenize(sentence)
        sentence_length = len(tokens)

        if current_length + sentence_length > max_length:
            # If the current segment plus new tokens exceeds max_length, start a new segment
            if current_segment:
                segments.append(tokenizer.convert_tokens_to_string(current_segment))
                current_segment = []
                current_length = 0

            # If the sentence itself exceeds max_length, split it directly
            if sentence_length > max_length:
                for i in range(0, sentence_length, max_length):
                    segments.append(tokenizer.convert_tokens_to_string(tokens[i:i + max_length]))
            else:
                # Otherwise, start the new segment with the current sentence
                current_segment = tokens
                current_length = sentence_length
        else:
            # Add tokens to the current segment
            current_segment.extend(tokens)
            current_length += sentence_length

    # Append the last segment if it exists
    if current_segment:
        segments.append(tokenizer.convert_tokens_to_string(current_segment))

    return segments

# Example usage
text = """Three months ended March 31, percent change at actual currency rates at constant currency rates (dollars in millions) 2023 2022 United States $9,201 $10,348 (11.1)% (11.1)%"""
segments = custom_split_text(text)
print(segments)


['three months ended march 31 , percent change at actual currency rates at constant currency rates ( dollars in millions ) 2023 2022 united states $ 9 , 201 $ 10 , 348 ( 11 . 1 ) % ( 11 . 1 ) %']


In [9]:
# tokenizer = BertTokenizer.from_pretrained("yiyanghkust/finbert-tone")

# def custom_split_text(text, max_length=508):
#     # Reserve space for special tokens
#     max_length -= 2

#     # Tokenize text into sentences
#     sentences = sent_tokenize(text)
#     segments = []

#     # Process each sentence
#     for sentence in sentences:
#         tokens = tokenizer.tokenize(sentence)
#         sentence_length = len(tokens)
        
#         if sentence_length > max_length:
#             # Split long sentence into chunks that fit into max_length
#             for i in range(0, sentence_length, max_length):
#                 segments.append(tokenizer.convert_tokens_to_string(tokens[i:i + max_length]))
#         else:
#             # Add the sentence as a segment
#             segments.append(tokenizer.convert_tokens_to_string(tokens))

#     return segments

# Example usage:
text = (
    "Once upon a time, in a land far, far away, there was a kingdom surrounded by towering mountains and deep, mystical forests. "
    "The kingdom was ruled by a wise and benevolent king who had three children. Each of his children was unique in their own way, "
    "and the king loved them all dearly. The eldest child, Prince Alexander, was a brave warrior who led the kingdom's armies with honor "
    "and courage. The second child, Princess Isabella, was known throughout the land for her beauty and her skill in the arts. She painted "
    "the most beautiful landscapes and composed music that could move anyone to tears. The youngest child, Prince Edward, was a scholar "
    "who spent his days in the grand library, studying the ancient texts and seeking knowledge about the world beyond their kingdom. "
    "One day, a mysterious traveler arrived at the kingdom's gates. He brought with him tales of strange lands and magical creatures. "
    "The traveler spoke of a distant realm where the trees were made of silver and the rivers flowed with liquid gold. Intrigued by these "
    "stories, the king invited the traveler to stay in the castle and share more of his adventures. As the days passed, the king's children "
    "grew more and more fascinated by the traveler's tales. Prince Alexander dreamed of embarking on a grand quest, Princess Isabella imagined "
    "painting the wondrous landscapes the traveler described, and Prince Edward longed to uncover the secrets of the distant realm's ancient "
    "knowledge. One evening, as the sun set behind the mountains, the traveler revealed that he possessed a magical map. This map, he claimed, "
    "could guide them to the realm of silver trees and golden rivers. The king, seeing the excitement in his children's eyes, decided to allow "
    "them to embark on this adventure. He believed it would be a journey that would help them grow and learn. With the traveler's guidance, the "
    "three siblings set out on their quest. They faced many challenges along the way. They crossed treacherous mountains, navigated dense forests, "
    "and sailed across vast oceans. Along their journey, they encountered mythical creatures and faced perilous trials that tested their courage, "
    "wisdom, and perseverance. Despite the hardships, the siblings' bond grew stronger, and they learned valuable lessons about themselves and each "
    "other. Finally, after many months of travel, they reached the realm described by the traveler. It was even more beautiful and enchanting than "
    "they had imagined. The trees sparkled like diamonds, and the rivers shimmered like molten gold. The air was filled with the sweet scent of "
    "blossoms, and the sound of birdsong echoed through the valleys. The three siblings spent time exploring this magical land, each finding what "
    "they had been searching for. Prince Alexander discovered a new purpose as he helped protect the realm from dark forces. Princess Isabella found "
    "endless inspiration for her art, creating masterpieces that captured the essence of the land's beauty. Prince Edward unearthed ancient scrolls "
    "and knowledge that expanded his understanding of the world and its mysteries. When the time came to return to their own kingdom, they did so "
    "with heavy hearts but a wealth of new experiences and wisdom. They had grown not only as individuals but also as a family. The king welcomed "
    "them back with open arms, proud of their accomplishments and the people they had become. The kingdom thrived under their leadership, and the "
    "stories of their adventure were passed down through generations, inspiring countless others to seek out their own paths and embrace the unknown. "
    "And so, the tale of the brave siblings and their journey to the magical realm became legend, a testament to the power of curiosity, bravery, "
    "and the bonds of family."
)
segments = custom_split_text(text)
segments

["once upon a time , in a land far , far away , there was a kingdom surrounded by towering mountains and deep , mystical forests . the kingdom was ruled by a wise and benevolent king who had three children . each of his children was unique in their own way , and the king loved them all dearly . the eldest child , prince alexander , was a brave warrior who led the kingdom ' s armies with honor and courage . the second child , princess isabella , was known throughout the land for her beauty and her skill in the arts . she painted the most beautiful landscapes and composed music that could move anyone to tears . the youngest child , prince edward , was a scholar who spent his days in the grand library , studying the ancient texts and seeking knowledge about the world beyond their kingdom . one day , a mysterious traveler arrived at the kingdom ' s gates . he brought with him tales of strange lands and magical creatures . the traveler spoke of a distant realm where the trees were made of s

In [10]:
# # Read the file and split the text by empty lines
# with open('../Data/Sec_Edgar_data/ABBV/2023-12-31.txt', 'r', encoding='utf-8') as file:
#     text = file.read()

# segments = custom_split_text(text)

# df = pd.DataFrame(segments, columns=['Segments'])

# df


In [11]:
# display(df.iloc[[4]])


In [12]:
sec_dict.values

array(['abandon', 'abandoned', 'abandoning', ..., 'yearend', 'yearends',
       'zeolite'], dtype=object)

In [13]:
for word in sec_dict:
    word_str = str(word)
    if word_str in text:
        print(word)

able
accomplish
accomplishment
accomplishments
beautiful
bling
challenge
challenges
dream
excitement
force
hardship
hardships
honor
ill
inspiration
leadership
lie
loss
peril
que
sec
sever
strong
stronger
uncover
valuable


In [14]:
# # Calculate word count for each feature
# word_counts = df.apply(lambda x: x.str.split().str.len())

# # Find the maximum word count across all features
# max_word_count = word_counts.max().max()

# print("Word counts for each feature:")
# print(word_counts)
# print("\nMaximum word count across all features:", max_word_count)


In [15]:
model = BertForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone",num_labels=3)

/home/arda/anaconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [16]:
# preds = []
# preds_proba = []
# embeddings = []

# tokenizer_kwargs = {"padding": 'max_length', "return_tensors": "pt", "max_length": 512}

# column = 'Segments'

# for i in range(len(df)):
#     text_content = df[column].iloc[i]
    
#     with torch.no_grad():
#         input_sequence = tokenizer(text_content, **tokenizer_kwargs)
        
#         logits = model(**input_sequence).logits
        
#         scores = {
#             k: v
#             for k, v in zip(
#                 model.config.id2label.values(),
#                 scipy.special.softmax(logits.numpy().squeeze()),
#             )
#         }
        
#         sentimentFinbert = max(scores, key=scores.get)
#         probabilityFinbert = max(scores.values())
        
#         preds.append(sentimentFinbert)
#         preds_proba.append(probabilityFinbert)

# df['Predicted Sentiment'] = preds
# df['Probability'] = preds_proba

# df

In [17]:
# preds = []
# preds_proba = []
# embeddings = []
# filtered_embeddings = {}
# num_of_words = []
# segment = []

# tokenizer_kwargs = {"padding": 'max_length', "return_tensors": "pt", "max_length": 512}

# column = 'Segments'

# stemmer = SnowballStemmer("english")
# stemmed_sec_dict = {stemmer.stem(word): word for word in sec_dict}

# for i in range(len(df)):
#     text_content = df[column].iloc[i]
    
#     with torch.no_grad():
#         input_sequence = tokenizer(text_content, **tokenizer_kwargs)
        
#         outputs = model(**input_sequence, output_hidden_states=True)
#         hidden_states = outputs.hidden_states

#         logits = outputs.logits
#         probabilities = scipy.special.softmax(logits.numpy().squeeze(), axis=0)
#         sentiment = model.config.id2label[probabilities.argmax()]

#         if (sentiment != "Neutral"):

#             preds.append(sentiment)
#             preds_proba.append(probabilities.max())
            
#             last_layer_embeddings = hidden_states[-1].numpy().squeeze()
#             print(last_layer_embeddings.shape)

#             token_ids = input_sequence["input_ids"].squeeze()
#             tokens = tokenizer.convert_ids_to_tokens(token_ids)
#             token_embeddings = {token: embedding for token, embedding in zip(tokens, last_layer_embeddings)}

#             segments_embeddings = []
#             finded_words = 0
#             for token, embedding in token_embeddings.items():
#                 stemmed_token = stemmer.stem(token)

#                 if stemmed_token in stemmed_sec_dict:
#                     original_word = stemmed_sec_dict[stemmed_token]
#                     filtered_embeddings[original_word] = embedding
#                     segments_embeddings.append(embedding)
#                     finded_words += 1

#             segments_embeddings = np.array(segments_embeddings)

#             if segments_embeddings.size == 0 or np.isnan(segments_embeddings).all():
#                 mean_embeddings = np.zeros(last_layer_embeddings.shape[1]) 
#             else:
#                 mean_embeddings = np.nanmean(segments_embeddings, axis=0)

#             embeddings.append(mean_embeddings)
#             num_of_words.append(finded_words)
#             segment.append(text_content)

# df_new = pd.DataFrame(segment, columns=['Segments'])
# df_new['Embeddings'] = embeddings
# df_new["Finded Words"] = num_of_words
# df_new['Predicted Sentiment'] = preds
# df_new['Probability'] = preds_proba
# df_new


In [18]:
# # Frequency of each sentiment
# sentiment_counts = df['Predicted Sentiment'].value_counts()
# print("Frequency of each sentiment:")
# print(sentiment_counts)

# # Proportion of each sentiment
# sentiment_proportions = df['Predicted Sentiment'].value_counts(normalize=True)
# print("\nProportion of each sentiment:")
# print(sentiment_proportions)

# # Average probability for the predictions
# average_probability = df['Probability'].mean()
# print("\nAverage confidence in predictions:")
# print(f"{average_probability:.2%}")

# # Assuming embeddings are stored in a list of numpy arrays
# all_embeddings = np.vstack(df['Embeddings'])  # Stack embeddings vertically

# # Calculate mean and standard deviation
# mean_embeddings = np.mean(all_embeddings, axis=0)
# std_embeddings = np.std(all_embeddings, axis=0)

# print("\nMean of embeddings:")
# print(mean_embeddings)

# print("\nStandard deviation of embeddings:")
# print(std_embeddings)

# # Assuming the rest of your code has populated `df` correctly as shown in previous messages
# print("Final DataFrame:")
# print(df)

# # Analyze predictions and embeddings
# print("Frequency of each sentiment:")
# print(df['Predicted Sentiment'].value_counts())

# print("\nProportion of each sentiment:")
# print(df['Predicted Sentiment'].value_counts(normalize=True))

# print("\nAverage confidence in predictions:")
# print(f"{df['Probability'].mean():.2%}")

# # For embeddings, assuming they are list of numpy arrays
# if df['Embeddings'].apply(len).sum() > 0:  # Check if there are any embeddings
#     all_embeddings = np.vstack(df['Embeddings'])
#     print("\nMean of embeddings:")
#     print(np.mean(all_embeddings, axis=0))
#     print("\nStandard deviation of embeddings:")
#     print(np.std(all_embeddings, axis=0))
# else:
#     print("\nNo embeddings available for analysis.")


In [19]:
# preds = []
# preds_proba = []
# embeddings = []
# filtered_embeddings = {}
# num_of_words = []

# tokenizer_kwargs = {"padding": 'max_length', "return_tensors": "pt", "max_length": 512}

# column = 'Segments'

# stemmer = SnowballStemmer("english")
# stemmed_sec_dict = {stemmer.stem(word): word for word in sec_dict}

# for i in range(len(df)):
#     text_content = df[column].iloc[i]
    
#     with torch.no_grad():
#         input_sequence = tokenizer(text_content, **tokenizer_kwargs)
        
#         # Assuming your model outputs logits as the first element of the output tuple
#         outputs = model(**input_sequence)
#         logits = outputs.logits  # Extract logits
#         preds.append(torch.argmax(logits, dim=-1).item())  # Append predicted class
#         preds_proba.append(torch.softmax(logits, dim=-1).squeeze().tolist())  # Append probabilities

#         hidden_states = outputs.hidden_states
#         last_layer_embeddings = hidden_states[-1].numpy().squeeze()

#         token_ids = input_sequence["input_ids"].squeeze()
#         tokens = tokenizer.convert_ids_to_tokens(token_ids)
#         token_embeddings = {token: embedding for token, embedding in zip(tokens, last_layer_embeddings)}

#         segments_embeddings = []
#         finded_words = 0
#         for token, embedding in token_embeddings.items():
#             stemmed_token = stemmer.stem(token)

#             if stemmed_token in stemmed_sec_dict:
#                 original_word = stemmed_sec_dict[stemmed_token]
#                 filtered_embeddings[original_word] = embedding
#                 segments_embeddings.append(embedding)
#                 finded_words += 1

#         segments_embeddings = np.array(segments_embeddings)

#         if segments_embeddings.size == 0 or np.isnan(segments_embeddings).all():
#             mean_embeddings = np.zeros(last_layer_embeddings.shape[1]) 
#         else:
#             mean_embeddings = np.nanmean(segments_embeddings, axis=0)

#         embeddings.append(mean_embeddings)
#         num_of_words.append(finded_words)

# df['Embeddings'] = embeddings
# df["Finded Words"] = num_of_words
# df['Predictions'] = preds
# df['Probabilities'] = preds_proba

# df


In [20]:
# import sys
# base_path = '../Data/Sec_Edgar_data'
# embedding_df = pd.read_csv("../Data/sentence_based_financeEmbeddings.csv")

# for ticker_folder in tqdm(os.listdir(base_path)):
#     ticker_path = os.path.join(base_path, ticker_folder)
    
#     if os.path.isdir(ticker_path):
#         ticker_name = ticker_folder
#         print(f"Ticker: {ticker_name}")

#         if ticker_name not in finance_ticker:
#             print(f"{ticker_name} not exists in the Financial Ticker. Continuing with the next ticker...")
#             continue

#         for file_name in os.listdir(ticker_path):

#             file_path = os.path.join(ticker_path, file_name)
#             file_name, extension = os.path.splitext(file_name)

#             with open(file_path, 'r') as file:
#                 print(f"Reading contents of {file_name}:")
#                 contents = file.read()

#                 segments = custom_split_text(contents)
#                 df = pd.DataFrame(segments, columns=['Segments'])

#                 if df.empty:
#                     print(f"df is empty {ticker_name}, {file_name}")
#                     sys.exit()

#                 preds = []
#                 preds_proba = []
#                 embeddings = []
#                 filtered_embeddings = {}
#                 num_of_words = []
#                 segment = []

#                 positive_embeddings = []
#                 negative_embeddings = []

#                 positive_prob = []
#                 negative_prob = []
#                 neutral_prob = []

#                 num_of_positive_sentences = 0
#                 num_of_negative_sentences = 0
#                 num_of_neutral_sentences = 0
                
#                 tokenizer_kwargs = {"padding": 'max_length', "return_tensors": "pt", "max_length": 512}

#                 column = 'Segments'

#                 stemmer = SnowballStemmer("english")
#                 stemmed_sec_dict = {stemmer.stem(word): word for word in sec_dict}

#                 for i in tqdm(len(df)):
#                     text_content = df[column].iloc[i]
                    
#                     with torch.no_grad():
#                         input_sequence = tokenizer(text_content, **tokenizer_kwargs)
                        
#                         outputs = model(**input_sequence, output_hidden_states=True)
#                         hidden_states = outputs.hidden_states

#                         logits = outputs.logits
#                         probabilities = scipy.special.softmax(logits.numpy().squeeze(), axis=0)
#                         sentiment = model.config.id2label[probabilities.argmax()]
#                         print(f"Sentiment: {probabilities}")

#                         if (sentiment != "Neutral"):

#                             preds.append(sentiment)
#                             preds_proba.append(probabilities.max())
                            
#                             last_layer_embeddings = hidden_states[-1].numpy().squeeze()
#                             print(last_layer_embeddings.shape)

#                             token_ids = input_sequence["input_ids"].squeeze()
#                             tokens = tokenizer.convert_ids_to_tokens(token_ids)
#                             token_embeddings = {token: embedding for token, embedding in zip(tokens, last_layer_embeddings)}

#                             segments_embeddings = []
#                             finded_words = 0
#                             for token, embedding in token_embeddings.items():
#                                 stemmed_token = stemmer.stem(token)

#                                 if stemmed_token in stemmed_sec_dict:
#                                     original_word = stemmed_sec_dict[stemmed_token]
#                                     filtered_embeddings[original_word] = embedding

#                                     segments_embeddings.append(embedding)
#                                     finded_words += 1

#                             segments_embeddings = np.array(segments_embeddings)

#                             if segments_embeddings.size == 0 or np.isnan(segments_embeddings).all():
#                                 mean_embeddings = np.zeros(last_layer_embeddings.shape[1]) 
#                             else:
#                                 mean_embeddings = np.nanmean(segments_embeddings, axis=0)

#                             embeddings.append(mean_embeddings)
#                             num_of_words.append(finded_words)
#                             segment.append(text_content)

#                 df_new = pd.DataFrame(segment, columns=['Segments'])
#                 df_new['Embeddings'] = embeddings
#                 df_new["Finded Words"] = num_of_words
#                 df_new['Predicted Sentiment'] = preds
#                 df_new['Probability'] = preds_proba

#                 positive_embeddings = df_new[df_new['Predicted Sentiment'] == "Positive"]["Embeddings"].values
#                 negative_embeddings = df_new[df_new['Predicted Sentiment'] == "Negative"]["Embeddings"].values
#                 num_of_negative = negative_embeddings.shape[0]
#                 num_of_positive = positive_embeddings.shape[0]

#                 embedding_dim = 768

#                 if positive_embeddings.size == 0:
#                     mean_embedding_positive = np.zeros(embedding_dim)
#                 else:
#                     mean_embedding_positive = np.mean(positive_embeddings, axis=0)

#                 if negative_embeddings.size == 0:
#                     mean_embedding_negative = np.zeros(embedding_dim)
#                 else:
#                     mean_embedding_negative = np.mean(negative_embeddings, axis=0)

#                 combined_mean_embeddings = np.concatenate([mean_embedding_positive, mean_embedding_negative])
#                 print(f"shape of the embedding: {mean_embedding_positive.shape}")
#                 print(f"combined shape of the embedding: {combined_mean_embeddings.shape}")
#                 column_names = [f'embedding_{i+1}' for i in range(combined_mean_embeddings.shape[0])]



#                 new_row = pd.DataFrame({
#                 'Ticker': [ticker_name], 
#                 'File Name': [file_name],
#                 **dict(zip(column_names, combined_mean_embeddings)),
#                 "Num Of Negative" : [num_of_negative],
#                 "Num Of Positive" : [num_of_positive]
#                 })
#                 embedding_df = pd.concat([embedding_df, new_row], ignore_index=True)
                

In [21]:
def contains_sec_dict_words(text, stemmed_sec_dict, stemmer):
    """ Check if the text contains any word from sec_dict """
    words = text.split()
    for word in words:
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_sec_dict:
            return True
    return False

In [22]:
import sys
base_path = '../Data/Sec_Edgar_data'
embedding_df = pd.read_csv("../Data/sentence_based_financeEmbeddings.csv")

for ticker_folder in tqdm(os.listdir(base_path)):
    ticker_path = os.path.join(base_path, ticker_folder)
    
    if os.path.isdir(ticker_path):
        ticker_name = ticker_folder
        print(f"Ticker: {ticker_name}")

        if ticker_name not in finance_ticker:
            print(f"{ticker_name} not exists in the Financial Ticker. Continuing with the next ticker...")
            continue

        for file_name in os.listdir(ticker_path):

            file_path = os.path.join(ticker_path, file_name)
            file_name, extension = os.path.splitext(file_name)

            if not embedding_df[(embedding_df['Ticker'] == ticker_name) & (embedding_df['Report Date'] == file_name)].empty:
                print(f"{ticker_name} and {file_name} already exist in embedding_df. Skipping this file...")
                continue

            with open(file_path, 'r') as file:
                print(f"Reading contents of {file_name}:")
                contents = file.read()

                segments = custom_split_text(contents)
                df = pd.DataFrame(segments, columns=['Segments'])

                if df.empty:
                    print(f"df is empty {ticker_name}, {file_name}")
                    sys.exit()

                segment = []

                positive_embeddings = []
                negative_embeddings = []

                positive_prob = []
                negative_prob = []
                neutral_prob = []

                sentiment_of_sentence = []

                num_of_positive_sentences = 0
                num_of_negative_sentences = 0
                num_of_neutral_sentences = 0
                
                found_words = 0
                
                tokenizer_kwargs = {"padding": 'max_length', "return_tensors": "pt", "max_length": 512}

                column = 'Segments'

                stemmer = SnowballStemmer("english")
                stemmed_sec_dict = {stemmer.stem(word): word for word in sec_dict}

                for i in tqdm(range(len(df))):
                    text_content = df[column].iloc[i]
                    # Check if the current sentence contains any word from sec_dict
                    # if not contains_sec_dict_words(text_content, stemmed_sec_dict, stemmer):
                    #     print("skipping the sentence.")
                    #     continue  # Skip this sentence if no word from sec_dict is found
                    
                    
                    with torch.no_grad():
                        input_sequence = tokenizer(text_content, **tokenizer_kwargs)
                        
                        outputs = model(**input_sequence, output_hidden_states=True)
                        hidden_states = outputs.hidden_states

                        logits = outputs.logits
                        probabilities = scipy.special.softmax(logits.numpy().squeeze(), axis=0)
                        sentiment = model.config.id2label[probabilities.argmax()]

                        if (sentiment == "Neutral"):
                            neutral_prob.append(probabilities[0])
                            positive_prob.append(probabilities[1])
                            negative_prob.append(probabilities[2])
                            num_of_neutral_sentences += 1

                        else:
                            neutral_prob.append(probabilities[0])
                            positive_prob.append(probabilities[1])
                            negative_prob.append(probabilities[2])

                            if sentiment == "Positive":
                                num_of_positive_sentences += 1
                            elif sentiment == "Negative":
                                num_of_negative_sentences += 1

                            sentiment_of_sentence.append(sentiment)
                            
                            last_layer_embeddings = hidden_states[-1].numpy().squeeze()

                            token_ids = input_sequence["input_ids"].squeeze()
                            tokens = tokenizer.convert_ids_to_tokens(token_ids)
                            token_embeddings = {token: embedding for token, embedding in zip(tokens, last_layer_embeddings)}

                            for token, embedding in token_embeddings.items():
                                stemmed_token = stemmer.stem(token)

                                if stemmed_token in stemmed_sec_dict:
                                    
                                    word_embedding = np.array(embedding)

                                    if sentiment == "Positive":
                                        positive_embeddings.append(word_embedding)

                                    elif sentiment == "Negative":
                                        negative_embeddings.append(word_embedding)

                                    found_words += 1

                            segment.append(" ".join(text_content))

                embedding_dim = 768

                positive_embeddings = np.array(positive_embeddings)
                negative_embeddings = np.array(negative_embeddings)

                if positive_embeddings.size == 0:
                    mean_embedding_positive = np.zeros(embedding_dim)
                else:
                    mean_embedding_positive = np.mean(positive_embeddings, axis=0)

                if negative_embeddings.size == 0:
                    mean_embedding_negative = np.zeros(embedding_dim)
                else:
                    mean_embedding_negative = np.mean(negative_embeddings, axis=0)

                combined_mean_embeddings = np.concatenate([mean_embedding_positive, mean_embedding_negative])
                print(f"shape of the embedding: {mean_embedding_positive.shape}")
                print(f"combined shape of the embedding: {combined_mean_embeddings.shape}")
                column_names = [f'embedding_{i+1}' for i in range(combined_mean_embeddings.shape[0])]

                new_row = pd.DataFrame({
                'Ticker': [ticker_name], 
                'Report Date': [file_name],
                "Num Of Negative" : [num_of_negative_sentences],
                "Num Of Positive" : [num_of_positive_sentences],
                "Found Words": [found_words],
                'Positive Prob': np.mean(positive_prob),
                'Negative Prob': np.mean(negative_prob),
                'Neutral Prob': np.mean(neutral_prob),
                **dict(zip(column_names, combined_mean_embeddings))
                })

                embedding_df = pd.concat([embedding_df, new_row], ignore_index=True)
                

  0%|          | 0/276 [00:00<?, ?it/s]

Ticker: TROW
TROW and 2012-03-31 already exist in embedding_df. Skipping this file...
TROW and 2017-06-30 already exist in embedding_df. Skipping this file...
TROW and 2018-06-30 already exist in embedding_df. Skipping this file...
TROW and 2020-12-31 already exist in embedding_df. Skipping this file...
TROW and 2015-09-30 already exist in embedding_df. Skipping this file...
TROW and 2016-06-30 already exist in embedding_df. Skipping this file...
TROW and 2016-09-30 already exist in embedding_df. Skipping this file...
TROW and 2019-09-30 already exist in embedding_df. Skipping this file...
TROW and 2019-12-31 already exist in embedding_df. Skipping this file...
TROW and 2015-06-30 already exist in embedding_df. Skipping this file...
TROW and 2020-06-30 already exist in embedding_df. Skipping this file...
TROW and 2016-12-31 already exist in embedding_df. Skipping this file...
TROW and 2017-03-31 already exist in embedding_df. Skipping this file...
TROW and 2014-12-31 already exist in e

  0%|          | 0/70 [00:00<?, ?it/s]

shape of the embedding: (768,)
combined shape of the embedding: (1536,)
Reading contents of 2017-12-31:


  0%|          | 0/82 [00:00<?, ?it/s]

...

shape of the embedding: (768,)
combined shape of the embedding: (1536,)
Reading contents of 2017-09-30:


  0%|          | 0/1 [00:00<?, ?it/s]

shape of the embedding: (768,)
combined shape of the embedding: (1536,)
Reading contents of 2019-03-31:


  0%|          | 0/1 [00:00<?, ?it/s]

shape of the embedding: (768,)
combined shape of the embedding: (1536,)


In [23]:
embedding_df.sort_values(by=["Report Date", "Ticker"])

,Ticker,Report Date,Num Of Negative,Num Of Positive,Found Words,Positive Prob,Negative Prob,Neutral Prob,embedding_1,embedding_2,...,embedding_1527,embedding_1528,embedding_1529,embedding_1530,embedding_1531,embedding_1532,embedding_1533,embedding_1534,embedding_1535,embedding_1536
125,DFS,2011-05-31,3.0,4.0,328.0,0.098141,0.057145,0.844714,-0.269836,-0.667547,...,0.132339,0.391349,-0.101878,0.289486,0.667223,0.351935,0.012379,-0.790106,-0.531331,0.280774
757,CBOE,2011-06-29,2.0,1.0,65.0,0.053857,0.078146,0.867997,0.035818,-0.497583,...,-0.112394,0.628557,1.008932,0.222512,0.212564,0.079795,-0.068103,-0.139793,-0.206865,0.666078
490,AFL,2011-06-30,2.0,10.0,444.0,0.185839,0.042638,0.771523,-0.445991,-0.376216,...,0.414589,0.791822,0.652430,0.275277,0.749954,-0.341811,-0.231431,-0.404139,0.103189,1.179630
1782,AON,2011-06-30,4.0,8.0,406.0,0.270472,0.141872,0.587656,-0.044927,-0.371451,...,0.351455,0.597015,0.459006,0.174953,0.796469,0.002994,-0.299642,-0.269401,-0.019401,0.684581
1686,AXP,2011-06-30,0.0,12.0,464.0,0.202702,0.007731,0.789567,-0.276930,-0.409391,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,TROW,2024-03-31,1.0,5.0,223.0,0.165471,0.032351,0.802178,-0.173672,-0.258428,...,0.554650,0.208854,0.025358,-0.128251,0.396882,0.363576,0.073009,-0.601066,-0.497182,0.161090
190,TRV,2024-03-31,7.0,9.0,630.0,0.183727,0.136209,0.680064,-0.120381,-0.499944,...,0.418682,0.656059,0.072387,0.074895,0.763338,0.024421,-0.134036,-0.454445,0.121910,0.733638
814,WFC,2024-03-31,0.0,0.0,0.0,0.000883,0.000037,0.999080,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
606,WRB,2024-03-31,0.0,2.0,84.0,0.085500,0.000328,0.914172,-0.170304,-0.138004,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
len(embedding_df["Ticker"].unique())

38

In [25]:
len(embedding_df["Ticker"].unique())

38

In [26]:
embedding_df

,Ticker,Report Date,Num Of Negative,Num Of Positive,Found Words,Positive Prob,Negative Prob,Neutral Prob,embedding_1,embedding_2,...,embedding_1527,embedding_1528,embedding_1529,embedding_1530,embedding_1531,embedding_1532,embedding_1533,embedding_1534,embedding_1535,embedding_1536
0,TROW,2012-03-31,0.0,3.0,96.0,0.433360,0.049688,0.516952,-0.220608,-0.126542,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,TROW,2017-06-30,0.0,3.0,118.0,0.129501,0.001244,0.869255,-0.518157,-0.086362,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,TROW,2018-06-30,1.0,4.0,164.0,0.111424,0.029975,0.858601,-0.349939,-0.147468,...,0.051092,0.533000,1.070406,0.323609,0.152008,0.558247,-0.725482,-0.463541,-0.755556,0.724856
3,TROW,2020-12-31,2.0,8.0,265.0,0.254435,0.053568,0.691996,-0.073542,-0.224136,...,0.145560,0.372795,0.258847,0.157887,-0.324517,-0.309704,0.183422,-0.203659,0.145782,0.195257
4,TROW,2015-09-30,2.0,1.0,105.0,0.099549,0.161237,0.739215,-0.388687,-0.327704,...,0.251360,0.669250,0.835729,0.199768,0.108205,0.292616,-0.445671,-0.225715,-0.427849,0.603479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1963,BK,2011-09-30,1.0,0.0,2.0,0.008405,0.983301,0.008294,0.000000,0.000000,...,0.459046,0.229575,0.993614,-0.849249,0.189553,0.800350,-0.806158,0.775358,-0.230105,0.429043
1964,BK,2018-03-31,1.0,0.0,2.0,0.008405,0.983301,0.008294,0.000000,0.000000,...,0.459046,0.229575,0.993614,-0.849249,0.189553,0.800350,-0.806158,0.775358,-0.230105,0.429043
1965,BK,2021-09-30,1.0,0.0,2.0,0.008405,0.983301,0.008294,0.000000,0.000000,...,0.459046,0.229575,0.993614,-0.849249,0.189553,0.800350,-0.806158,0.775358,-0.230105,0.429043
1966,BK,2017-09-30,1.0,0.0,2.0,0.008405,0.983301,0.008294,0.000000,0.000000,...,0.459046,0.229575,0.993614,-0.849249,0.189553,0.800350,-0.806158,0.775358,-0.230105,0.429043


In [29]:
embedding_df.to_csv("../Data/sentence_based_financeEmbeddings.csv", index=False)

In [28]:
embedding_df

,Ticker,Report Date,Num Of Negative,Num Of Positive,Found Words,Positive Prob,Negative Prob,Neutral Prob,embedding_1,embedding_2,...,embedding_1527,embedding_1528,embedding_1529,embedding_1530,embedding_1531,embedding_1532,embedding_1533,embedding_1534,embedding_1535,embedding_1536
0,TROW,2012-03-31,0.0,3.0,96.0,0.433360,0.049688,0.516952,-0.220608,-0.126542,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,TROW,2017-06-30,0.0,3.0,118.0,0.129501,0.001244,0.869255,-0.518157,-0.086362,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,TROW,2018-06-30,1.0,4.0,164.0,0.111424,0.029975,0.858601,-0.349939,-0.147468,...,0.051092,0.533000,1.070406,0.323609,0.152008,0.558247,-0.725482,-0.463541,-0.755556,0.724856
3,TROW,2020-12-31,2.0,8.0,265.0,0.254435,0.053568,0.691996,-0.073542,-0.224136,...,0.145560,0.372795,0.258847,0.157887,-0.324517,-0.309704,0.183422,-0.203659,0.145782,0.195257
4,TROW,2015-09-30,2.0,1.0,105.0,0.099549,0.161237,0.739215,-0.388687,-0.327704,...,0.251360,0.669250,0.835729,0.199768,0.108205,0.292616,-0.445671,-0.225715,-0.427849,0.603479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1963,BK,2011-09-30,1.0,0.0,2.0,0.008405,0.983301,0.008294,0.000000,0.000000,...,0.459046,0.229575,0.993614,-0.849249,0.189553,0.800350,-0.806158,0.775358,-0.230105,0.429043
1964,BK,2018-03-31,1.0,0.0,2.0,0.008405,0.983301,0.008294,0.000000,0.000000,...,0.459046,0.229575,0.993614,-0.849249,0.189553,0.800350,-0.806158,0.775358,-0.230105,0.429043
1965,BK,2021-09-30,1.0,0.0,2.0,0.008405,0.983301,0.008294,0.000000,0.000000,...,0.459046,0.229575,0.993614,-0.849249,0.189553,0.800350,-0.806158,0.775358,-0.230105,0.429043
1966,BK,2017-09-30,1.0,0.0,2.0,0.008405,0.983301,0.008294,0.000000,0.000000,...,0.459046,0.229575,0.993614,-0.849249,0.189553,0.800350,-0.806158,0.775358,-0.230105,0.429043
